In [26]:
import threading
import time
from queue import Queue
import copy

In [6]:
threading.active_count()

5

In [7]:
threading.enumerate()

[<_MainThread(MainThread, started 15592)>,
 <Thread(Thread-6, started daemon 19644)>,
 <Heartbeat(Thread-7, started daemon 16312)>,
 <ControlThread(Thread-5, started daemon 29312)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 20588)>]

In [8]:
threading.current_thread()

<_MainThread(MainThread, started 15592)>

# 1

In [10]:
def thread_job():
    print('This is a thread of %s' % threading.current_thread())

In [11]:
thread = threading.Thread(target=thread_job,)
thread.start()

This is a thread of <Thread(Thread-8, started 24552)>


# 2 join to main 

In [18]:
def thread_job():
    print('T1 start\n')
    for i in range(10):
        time.sleep(0.1)
    print('T1 finish\n')
def T2_job():
    print('T2 start\n')
    print('T2 finish\n')

In [19]:
thread1 = threading.Thread(target=thread_job, name='T1')
thread2 = threading.Thread(target=T2_job, name='T2')
thread1.start()
thread2.start()
thread2.join()
thread1.join()
print('all finish')

T1 start

T2 start

T2 finish

T1 finish

all finish


In [20]:
thread1 = threading.Thread(target=thread_job, name='T1')
thread2 = threading.Thread(target=T2_job, name='T2')
thread1.start()
thread2.start()
# thread2.join()
# thread1.join()
print('all finish')

T1 start

T2 start

T2 finish

all finish
T1 finish



# 3 to Queue

In [23]:
def job(l,q):
    for i in range (len(l)):
        l[i] = l[i]**2
    q.put(l)

In [24]:
def multithreading():
    q =Queue()
    threads = []
    data = [[1,2,3],[3,4,5],[4,4,4],[5,5,5]]
    for i in range(4):
        t = threading.Thread(target=job,args=(data[i],q))
        t.start()
        threads.append(t)
    for thread in threads:
        thread.join()
    results = []
    for _ in range(4):
        results.append(q.get())
    print(results)

In [25]:
multithreading()

[[1, 4, 9], [9, 16, 25], [16, 16, 16], [25, 25, 25]]


# 4 Global Interpreter Lock (GIL)

In [27]:
def job(l, q):
    res = sum(l)
    q.put(res)

In [28]:
def multithreading(l):
    q = Queue()
    threads = []
    for i in range(4):
        t = threading.Thread(target=job, args=(copy.copy(l), q), name='T%i' % i)
        t.start()
        threads.append(t)
    [t.join() for t in threads]
    total = 0
    for _ in range(4):
        total += q.get()
    print(total)

In [32]:
l = list(range(1000000))
s_t = time.time()
normal(l*4)
print('normal: ',time.time()-s_t)

1999998000000
normal:  0.08159685134887695


In [33]:
s_t = time.time()
multithreading(l)
print('multithreading: ', time.time()-s_t)

1999998000000
multithreading:  0.09496140480041504


# 5 Lock

In [36]:
def job1():
    global A,lock
    lock.acquire()
    for i in range(10):
        A+=1
        print('job1',A)
    lock.release()

In [37]:
def job2():
    global A,lock
    lock.acquire()
    for i in range(10):
        A+=10
        print('job2',A)
    lock.release()

In [40]:
lock=threading.Lock()
A=0
t1=threading.Thread(target=job1)
t2=threading.Thread(target=job2)
t1.start()
t2.start()
t1.join()
t2.join()

job1 1
job1 2
job1 3
job1 4
job1 5
job1 6
job1 7
job1 8
job1 9
job1 10
job2 20
job2 30
job2 40
job2 50
job2 60
job2 70
job2 80
job2 90
job2 100
job2 110
